In [1]:
import geopandas as gpd

# Load the shapefiles
taxi_zones = gpd.read_file('../data/taxi_zones/taxi_zones.shp')
precincts = gpd.read_file('../data/Police Precincts/geo_export_da8421e8-1e0e-441f-8db9-87a09ccc7b8b.shp')

In [2]:
# Check the CRS of both shapefiles
print(taxi_zones.crs)
print(precincts.crs)

taxi_zones = taxi_zones.to_crs(precincts.crs)

EPSG:2263
EPSG:4326


In [9]:
# Calculate the centroids of the taxi zones
taxi_zones['centroid'] = taxi_zones.geometry.centroid

# Create a GeoDataFrame for the centroids, including LocationID
centroids = gpd.GeoDataFrame(taxi_zones[['LocationID', 'centroid']], geometry='centroid', crs=taxi_zones.crs)

# Perform spatial join where each centroid is matched to a precinct
taxi_to_precinct = gpd.sjoin(centroids, precincts, how="left", predicate="within")

# Optionally, drop the centroid column and keep only relevant columns (e.g., LocationID and precinct)
taxi_to_precinct = taxi_to_precinct[['LocationID', 'precinct']]  # Replace 'precinct_id' with the actual precinct column name
# Fill NaN values with a placeholder value (e.g., 0) before conversion to int
taxi_to_precinct['precinct'] = taxi_to_precinct['precinct'].fillna(0).astype(int)

filtered_gdf = taxi_to_precinct[taxi_to_precinct['precinct'] == 0]
print(filtered_gdf)


     LocationID  precinct
0             1         0
45           46         0
103         103         0


/var/folders/dm/xmt0kc7d45bbjbmvfs9tpy1r0000gn/T/ipykernel_27466/2300947001.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taxi_zones['centroid'] = taxi_zones.geometry.centroid


In [6]:
# Save the resulting mapping to a new CSV file
taxi_to_precinct.to_csv("../data/taxi_zone_precinct_mapping.csv", index=False)